In [3]:
import dash
import dash_leaflet
import plotly.express as px
import pandas as pd
print("All imports OK")

All imports OK


In [4]:
# Setup the Jupyter version of Dash
from dash import Dash

# Dash and related imports
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output, State
import plotly.express as px

# Other libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import your CRUD class
from crud_module_enhanced import CRUD


###########################
# Data Manipulation / Model
###########################

MONGO_URI = os.getenv("MONGO_URI")  # optional; supported via CRUD.from_env()

#  Milestone Four: Dashboard should be read-only (least privilege).
# Admin CRUD is handled separately (admin script or notebook).
db = CRUD.from_env(
    prefix="MONGO",
    read_only=True,
    db_name="aac",
    collection="animals",
)

# Optional sanity check (helps catch connection issues early)
if not db.ping():
    raise RuntimeError("MongoDB connection failed. Check MONGO_* environment variables and MongoDB service status.")

# Retrieve all documents initially (exclude MongoDB internal _id)
df = pd.DataFrame.from_records(db.read({}, projection={"_id": 0}))


#########################
# Dashboard Layout / View
#########################

from pathlib import Path
import base64

encoded_image = base64.b64encode(
    Path("Grazioso Salvare Logo.png").read_bytes()
).decode()

header = html.Div(
    children=[
        html.Img(
            src=f"data:image/png;base64,{encoded_image}",
            style={"height": "100px", "display": "block", "margin": "0 auto"},
        ),
        html.H1("Grazioso Salvare Dashboard", style={"textAlign": "center", "marginTop": "10px"}),
    ],
    style={"marginBottom": "10px"},
)

app = Dash(__name__)

app.layout = html.Div([
    header,
    html.Hr(),

    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.RadioItems(
            id="filter-type",
            options=[
                {"label": "Reset", "value": "RESET"},
                {"label": "Water Rescue", "value": "WATER"},
                {"label": "Mountain or Wilderness Rescue", "value": "MOUNTAIN"},
                {"label": "Disaster or Individual Tracking", "value": "DISASTER"},
            ],
            value="RESET",
            labelStyle={"display": "inline-block", "margin-right": "20px"},
        ),
    ]),
    html.Hr(),

    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict("records"),
        page_size=10,
        style_table={"overflowX": "auto"},
        row_selectable="single",
        selected_rows=[0],
        sort_action="native",
        filter_action="native",
    ),

    html.Br(),
    html.Hr(),

    html.Div(
        style={"display": "flex"},
        children=[
            html.Div(id="graph-id", style={"flex": "1"}),
            html.Div(id="map-id", style={"flex": "1"}),
        ],
    ),
])


#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'RESET':
        query = {}
    elif filter_type == 'WATER':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever"]}}
    elif filter_type == 'MOUNTAIN':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute"]}}
    elif filter_type == 'DISASTER':
        query = {"breed": {"$in": ["Doberman Pinscher", "Golden Retriever", "Border Collie"]}}
    else:
        query = {}

    records = db.read(query, projection={"_id": 0})
    dff = pd.DataFrame.from_records(records)
    return dff.to_dict('records')


@app.callback(
    Output("graph-id", "children"),
    [Input("filter-type", "value")]
)
def update_graphs(filter_type):
    if filter_type == "RESET":
        query = {}
    elif filter_type == "WATER":
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever"]}}
    elif filter_type == "MOUNTAIN":
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute"]}}
    elif filter_type == "DISASTER":
        query = {"breed": {"$in": ["Doberman Pinscher", "Golden Retriever", "Border Collie"]}}
    else:
        query = {}

    breed_counts = db.top_breeds(match_query=query, limit=10)
    if not breed_counts:
        return "No Data Available"

    dff_count = pd.DataFrame(breed_counts)
    fig = px.pie(dff_count, names="breed", values="count", title="Top 10 Breeds")
    return [dcc.Graph(figure=fig)]


@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [
        Input('datatable-id', "derived_virtual_data"),
        Input('datatable-id', "derived_virtual_selected_rows")
    ]
)
def update_map(viewData, selected_rows):
    if not viewData:
        return "No Data Available"

    dff = pd.DataFrame.from_dict(viewData)
    row = selected_rows[0] if selected_rows else 0

    if len(dff.columns) < 15:
        return "Not enough columns for lat/long."

    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[32.7767, -96.7970],
            zoom=10,
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                    children=[
                        dl.Tooltip(str(dff.iloc[row, 4])),
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(str(dff.iloc[row, 9]))
                        ])
                    ]
                )
            ]
        )
    ]


if __name__ == '__main__':
    app.run(debug=True, port=8050)


---------------------------------------------------------------------------
ClientDisconnected                        Traceback (most recent call last)
ClientDisconnected: 400 Bad Request: The browser (or proxy) sent a request that this server could not understand.

